In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import time
import datetime
import os
import configparser 

config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
slack_token = config['DEFAULT']['Slack-Token']
channel = config['DEFAULT']['Channel']
file_path = config['DEFAULT']['File-Path']
log_file_path = config['DEFAULT']['Log-File-Path']
out_folder = config['DEFAULT']['Out-Folder']

import pymongo
from pymongo import MongoClient
client = MongoClient(ip, int(port))

from slacker import Slacker
slack = Slacker(slack_token)

KeyError: 'IP'

In [ ]:
def connect_database(client,slack):
    try:
        # connect to database
        db_twitter = client["Twitter"]
        collections_twitter = db_twitter.collection_names()
    except Exception as e:
        slack.chat.post_message(channel,str(e))
        with open(log_file_path,"a+") as output_file:
            output_file.write("---------------------------"+"\n")
            output_file.write(str(datetime.datetime.now())+"\n")
            output_file.write(str(e) + "\n")
    return db_twitter, collections_twitter

In [2]:
def current_year_week():
    # get current year and current week number
    current_timestamp = int(time.time() * 1000)
    current_year = int(datetime.datetime.now().year)
    print("current year : " + str(current_year))

    current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
    print("current week : " + str(current_week))
    return current_year, current_week

In [3]:
# create foler if not exist
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
def get_lastest_number(db_twitter,collections_twitter,current_year,current_week):
    w = "W"+str(current_week)
    dic_collection = {}
    for i in collections_twitter:
        if i.startswith(str(current_year)) and w in i:
            dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
    return dic_collection

In [ ]:
def read_file(file_path):
    with open(file_path,"a+") as input_file:
        # back to first line
        input_file.seek(0)
        line = input_file.readlines()
    return line

In [ ]:
def get_old_number(line):
    old_dic_collection = {}
    for i in line:
        i = i.replace("\n","")
        collection_name = i.split(":")[0]
        document_number = i.split(":")[1]
        old_dic_collection[collection_name] = "{:}".format(document_number)
    return old_dic_collection

In [ ]:
def send_msg_slack(dic_collection,old_dic_collection,slack):
    for collection in dic_collection:
        if collection in old_dic_collection:
            if int(dic_collection[collection]) == int(old_dic_collection[collection]):  
                # bold message
                message = "[" + collection + "] is the same last run: " + '\033[1m' + dic_collection[collection] + '\033[0m'
                slack.chat.post_message(channel,message)
            elif int(dic_collection[collection]) < int(old_dic_collection[collection]):  
                # bold message
                message = "[" + collection + "] is reduced last run: " + '\033[1m' + dic_collection[collection] + '\033[0m'
                slack.chat.post_message(channel,message)
                

In [ ]:
def write_file(dic_collection, file_path):
    with open(file_path,"w+") as output_file:
        for key in sorted(dic_collection): 
            output_file.write("%s: %s" % (key, dic_collection[key])+"\n") 
    print("The result file is ready.")

In [ ]:
def write_log_file(dic_collection, log_file_path):
    with open(log_file_path,"a+") as output_file:
        output_file.write("---------------------------"+"\n")
        output_file.write(str(datetime.datetime.now())+"\n")
        for key in sorted(dic_collection): 
            output_file.write("%s: %s" % (key, dic_collection[key])+"\n") 
    print("The log file is ready.")

In [ ]:
create_folder(out_folder)

try:
    db_twitter, collections_twitter = connect_database(client,slack)
except:
    pass
else:
    current_year, current_week = current_year_week()

    dic_collection = get_lastest_number(db_twitter,collections_twitter,current_year,current_week)

    line = read_file(file_path)

    old_dic_collection = get_old_number(line)

    send_msg_slack(dic_collection,old_dic_collection,slack)

    write_file(dic_collection, file_path)  
    write_log_file(dic_collection, log_file_path)